In [10]:
import numpy as np
import sys,os

from sklearn.model_selection import learning_curve
sys.path.append(os.pardir)
from common.layers import *
from common.gradient import numerical_gradient
from collections import OrderedDict

# 基于误差反向传播实现Relu激活函数
class Relu:
    def __init__(self):
        self.mask = None
    
    ## 正向传播算法
    def forward(self,x):
        self.mask = (x <= 0)  ## 所有x <= 0的地方都置为True
        out = x.copy()  ## 赋值一份数据
        out[self.mask] = 0  ## 所有x <= 0 的地方 正向传播为0
        return out

    ## 反向传播算法
    def backward(self,out):
        ## 根据如果正向传播时输入的x大于0  那么反向传播会将上游的值原封不动的传递给下游
        ## 如果正向传播时x小于等于0，则反向传播中传给下游的信号将停在此处
        dout = out[self.mask] = 0 ## 小于或者等于x的地方直接置为0
        dx = dout
        return dx
    
## 实现仿射变换层
class Affline:
    def __init__(self,W,b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None

    ## 前向传播
    def forward(self,x):
        self.x = x
        out = np.dot(x,self.W) + self.b
        return out

    ## 反向传播算法
    def backward(self,dout):
        dx = np.dot(dout,self.W.T)
        self.dw = np.dot(self.x.T,dout)
        self.db = np.sum(dout,axis = 0)

        return dx


## 输出层激活函数  输出概率分布
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a - c)  ## 溢出对策
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a



def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 监督数据是one-hot-vector的情况下，转换为正确解标签的索引
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

class SoftmaxWithLoss:

    def __init__(self):
        self.loss = None  ## 损失
        self.y = None  # softmax的输出
        self.t = None  # 标签数据 独热编码

    ## 这里的x已经经过前面的放射变换 得到的结果
    def forward(self,x,t):
        self.t = t
        self.y = softmax(x)  ## 输出概率  这里的x其实就是  经过前面若干隐藏层激活曾得到的结果
        ## 计算损失  交叉熵损失函数
        self.loss = cross_entropy_error(self.y,self.t)
        return self.loss

    def backward(self,dout = 1):
        ## dout 默认参数是1
        batch_size = self.t.shape[0]  ## 计算批次大小
        dx = (self.y - self.t) / batch_size  ## 计算反向传播误差  单个数据  平均误差
        return dx


class TwoLayerNet:

    ## 初始化参数
    def __init__(self,input_size,hidden_size,output_size,weight_init_std = 0.01):
    
        ## 初始化权重参数 高斯分布随机生成参数
        self.params = {}## 保存参数的字典
        self.params['W1'] = weight_init_std * np.random.randn(input_size,hidden_size)
        self.params['b1'] = np.zeros(hidden_size) ## 大小为隐藏层神经元个数
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size,output_size)
        self.params['b2'] = np.zeros(output_size)


        ## 生成层
        self.layers = OrderedDict()  ## 创建一个有序字典
        self.layers['Affine1'] = Affine(self.params['W1'],self.params['b1']) ## 创建第一个隐藏层
        self.layers['Relu1'] = Relu()  ## 创建第一个Relu激活层
        self.params['Affine2'] = Affine(self.params['W2'],self.params['b2'])  ## 创建输出层

        ## 上面的输出结果 经过softmax输出概率 然后计算交叉熵损失 

        self.lastLayer = SoftmaxWithLoss()

    ## 对于每一个层正向传播计算输出结果  直到最后一个隐藏层或者激活层
    def predict(self,x):
        for layer in self.layers.values():
            x = layer.forward(x)

        return x

    def loss(self,x,t):
        ## 计算损失函数

        ## 先通过predict函数计算正向传播结果  然后再调用lastLayer的forward函数计算正向传播的softmax概率以及损失
        y = self.predict(x)

        return self.lastLayer.forward(y,t)  # softmax 输出 概率  然后计算交叉熵损失


    def accuracy(self,x,t):
        ## 计算预测的精确度
        y = self.predict(x)  # 先计算预测值
        y = np.argmax(y,axis = 1) ## 获取每一层的最大值下标

        if t.ndim != 1 : 
            t = np.argmax(t,axis = 1)

        accuracy = np.sum(y == t) / float(x.shape[0])
        return accuracy


    ## x： 输入数据   t: 监督数据
    ## 基于数值微分的误差传播算法

    def numerical_gradient(self,x,t):
        loss_W = lambda W:self.loss(x,t)

        grads = {}
        grads['W1'] = numerical_gradient(loss_W,self.params['W1'])
        grads['b1'] = numerical_gradient(loss_W,self.params['b1'])
        grads['W2'] = numerical_gradient(loss_W,self.params['W2'])
        grads['b2'] = numerical_gradient(loss_W,self.params['b2'])

        return grads

    def gradient(self,x,t):
        # 误差反向传播算法
        self.loss(x,t)  ## 首先计算损失:先预测  softmax 然后交叉熵损失

        dout = 1
        dout = self.lastLayer.backward(dout)  ## 计算反向传播误差

        ## 取出神经网络每一个层 然后逆序
        layers = list(self.layers.values())
        layers.reverse() ## 逆序

        ## 调用神经网络中 隐藏层 激活岑的反向传播函数
        for layer in layers:
            dout = layer.backward(dout)


        ## 存储误差

        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db


        return grads  

from dataset.mnist import load_mnist


## 读取数据
(x_train,t_train),(x_test,t_test) = load_mnist(normalize=True,one_hot_label = True)
# print(x_train)

network = TwoLayerNet(input_size = 784,hidden_size=50,output_size=10)

iters_num = 10000 ## 迭代次数10000

train_size = x_train.shape[0]  ## 获取有多少张图片
batch_size = 100 ## 批量大小
learning_rate = 0.1  ##学习率
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size,1)  ## 计算多少批次

for i in range(iters_num):
    ## 随机获取下标序号
    batch_mask = np.random.choice(train_size,batch_size)  ## 随机获取批量大小的数据

    ## 获取批量大小的训练数据以及相应的标签
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]

    ## 通过误差反向传播算法计算误差
    grad = network.gradient(x_batch,t_batch)

    ## 更新神经网络参数
    for  key in ('W1','b1','W2','b2'):
        network.params[key] -= learning_rate * grad[key]

    loss = network.loss(x_batch,t_batch)
    train_loss_list.append(loss)

    ## 收集 每处理一个批次 收集一下 训练集以及测试集的精确度
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train,t_train)
        train_acc_list.append(train_acc)

        test_acc = network.accuracy(x_test,t_test)
        test_acc_list.append(test_acc)

        print(train_acc,test_acc)

AttributeError: 'NoneType' object has no attribute 'ndim'